In [4]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

In [31]:
# Load the CSV Data
df= pd.read_csv('../data/handlebar/Accelerometer_data_merged_ESP1.csv')  # Load the CSV data using pandas
# Filter the DataFrame to include only the necessary columns
df_filtered = df[['NTP','Acc-Z','curb_scene']]
df_filtered.head()

,NTP,Acc-Z,curb_scene
0,2024-07-24 10:26:01.863,8.762772,0.0
1,2024-07-24 10:26:01.867,8.753204,0.0
2,2024-07-24 10:26:01.872,8.762772,0.0
3,2024-07-24 10:26:01.877,8.781937,0.0
4,2024-07-24 10:26:01.884,8.753204,0.0


In [32]:
def handle_missing_values_length(df_filtered):
    # Handle missing values length
    missing_info = {}  # List to store (start_index, current_count)
    current_count = 0
    start_index = None  # To store the start time of missing values
    for index, row in df_filtered.iterrows():
        if np.isnan(row['Acc-Z']):
            if start_index is None:
                start_index = index
                current_count += 1
            if start_index is not None:
                current_count += 1
        # If the value is not NaN, continue sequence broke
        else:
            if start_index is not None:
                missing_info[start_index] = current_count
                start_index = None
                current_count = 0

    # Handle case where the last segment has missing values
    if current_count > 0:
        missing_info[start_index] = current_count

    for key, value in missing_info.items():
        print(key, value)
        if value > 100:
            print("Start Index:", key, "Count:", value)  
    print("No missing value longer than half a second")

    return missing_info


In [ ]:
missing_info = handle_missing_values_length(df_filtered)

21827 2
21832 2
21855 2
21860 2
21877 2
21884 2
21913 2
21931 2
21938 2
21946 2
21966 2
21971 2
23762 3
23785 2
23788 2
23821 2
23840 2
23854 2
23861 2
23882 2
23884 2
29727 2
29731 2
29768 2
29773 2
29787 2
29798 2
29803 2
29806 2
29829 2
29849 2
31426 2
31429 2
31449 2
31468 2
31483 2
31485 2
31488 2
31506 2
31514 2
31518 2
31538 2
31540 2
33691 2
33694 2
33709 2
33724 2
33729 2
33740 2
33752 2
33756 2
33772 2
33784 2
33789 2
35263 2
35266 2
35282 2
35299 2
35302 2
35315 2
35319 2
35329 2
35346 2
35362 2
35366 2
37547 2
37550 2
37565 2
37580 2
37585 2
37594 2
37605 2
37610 2
37621 2
37628 2
37644 2
39101 2
39104 2
39117 2
39120 2
39133 2
39143 2
39146 2
39148 2
39161 2
39167 2
39186 2
41417 2
41422 2
41437 2
41442 2
41455 2
41458 2
41479 2
41482 2
41487 2
41497 2
41505 2
41518 2
41521 2
43033 2
43049 2
43052 2
43063 2
43066 2
43069 2
43080 2
43094 2
43099 2
43118 2
45302 2
45304 2
45322 2
45325 2
45336 2
45351 2
45360 2
45364 2
45368 2
45377 2
45384 2
45401 2
46944 2
46947 2
46963 2


In [35]:
df.loc[21827]

Unnamed: 0                    21827
NTP                             NaN
Acc-X                           NaN
Acc-Y                           NaN
Acc-Z                           NaN
Timestamp_unix      1721816870306.0
curb_activity                   1.0
curb_scene                      1.0
curb_height_down                1.0
curb_height_up                  3.0
curb_type_down                  1.0
curb_type_up                    2.0
roughness                      -1.0
Name: 21827, dtype: object

In [ ]:
# TODO:handle missing values
# make a copy of the dataframe
# If the upper row have the same label in curb_scene, then fill the missing value with the same value in 'ACC-Z', 'NTP + 5'00-DataExploration.ipynb
# else: fill the missing value with the other side below with the next not nan value

for index, row in df_filtered.iterrows():
    if np.isnan(row['Acc-Z']):
        if index-1 >= 0:
            if df_filtered.loc[index-1]['curb_scene'] == df_filtered.loc[index]['curb_scene']:
                df_filtered.at[index, 'Acc-Z'] = df_filtered.loc[index-1]['Acc-Z']
            else:
             # Find the next non-NaN value below
                for j in range(index + 1, len(df_filtered)):
                    if not pd.isnull(df_filtered['Acc-Z'].iloc[j]):
                        df_filtered['Acc-Z'].iloc[index] = df_filtered['Acc-Z'].iloc[j]
                        break

C:\Users\liuzi\AppData\Local\Temp\ipykernel_9428\4249379710.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_filtered['Acc-Z'].iloc[index] = df_filtered['Acc-Z'].iloc[j]
C:\Users\liuzi\AppData\Local\Temp\ipykernel_9428\4249379710.py:1

In [47]:
missing_info_df_filtered = handle_missing_values_length(df_filtered)
print(missing_info_df_filtered)

No missing value longer than half a second
{}


In [50]:
print(df_filtered.loc[41458 -10:41458 +5])


                           NTP      Acc-Z  curb_scene
41448  2024-07-24 10:29:27.343   7.058105         1.0
41449  2024-07-24 10:29:27.348   8.638275         1.0
41450  2024-07-24 10:29:27.352  13.814545         1.0
41451  2024-07-24 10:29:27.358  13.737930         1.0
41452  2024-07-24 10:29:27.361  12.650955         1.0
41453  2024-07-24 10:29:27.368  11.118668         1.0
41454  2024-07-24 10:29:27.373   6.971909         1.0
41455                      NaN   6.971909         1.0
41456  2024-07-24 10:29:27.377   7.388504         1.0
41457  2024-07-24 10:29:27.381   8.465897         1.0
41458                      NaN   8.465897         1.0
41459  2024-07-24 10:29:27.387   6.440399         1.0
41460  2024-07-24 10:29:27.391   7.599197         1.0
41461  2024-07-24 10:29:27.396   4.582504         1.0
41462  2024-07-24 10:29:27.401   5.717361         1.0
41463  2024-07-24 10:29:27.408   8.925583         1.0
